In [ ]:
TESTS_TO_GENERATE_FILE_NAME = "./tests_to_generate.csv"
POSSIBLE_JSON_ROUTES_FILE_NAME = "../Server/Server.Tests/possibleJsonRoutes.txt"
STATIONS_XML_FILE_NAME = "../Server/Server/Stations.xml"

In [ ]:
first_line = True
raw_tests = []

with open(TESTS_TO_GENERATE_FILE_NAME) as f:
  lines = list(
    map(
        lambda x: x.replace("\n", "").strip(),
        f.readlines()
      )
  )

for line in lines:
  if first_line:
    first_line = False
    continue

  raw_tests.append(tuple(line.split(",")))

tests_to_generate = {}
for test_tuple in raw_tests:
  tests_to_generate.setdefault(test_tuple[2], []).append((test_tuple[0], test_tuple[1]))


In [ ]:

with open(POSSIBLE_JSON_ROUTES_FILE_NAME) as f:
  lines = list(
    map(
      lambda x: x.replace("\n", "").strip(),
      f.readlines()
    )
  )

generated_test_index = len(lines) + 1
print(f"Starting index for generated tests: {generated_test_index}")


In [ ]:
CSHARP_TEMPLATE = """        Assert.Equal(routes[{0}],_commands.RouteMe(_commands.getStationById({1}), _commands.getStationById({2}))?.ToString()); // {3} to {4}"""

TEST_ORDERING = [
  "Travel one change one",
  "Travel many change one",
  "Travel one change many",
  "2 Changes",
  "Express",
  "Random Long Routes"
]

In [ ]:
from graph_stations import GraphBuilder
import json

In [ ]:

builder = GraphBuilder()
stations_by_id = builder.build_graph(STATIONS_XML_FILE_NAME, [], True)
stations, nodes, edges = builder.dijkstra("273", "461", stations_by_id)
print("N", nodes)
instructions = builder.route_nodes_to_instructions(nodes, include_on_off_ids=True)
instructions


In [ ]:
builder = GraphBuilder()

stations_by_id = builder.build_graph("../Server/Server/Stations.xml", [], True)
csharp_output_lines = []
possible_routes_output_lines = []
for test_type in TEST_ORDERING:
  csharp_output_lines.append("")
  csharp_output_lines.append(f"        /* ---------------------- {test_type} required tests. ---------------------- */")
  csharp_output_lines.append("")
  tests = tests_to_generate[test_type]
  for test in tests:
    
    start_station_id = test[0]
    start_station_name = stations_by_id[start_station_id].name
    end_station_id = test[1]
    end_station_name = stations_by_id[end_station_id].name
    print(f"Generating Test and expected results for: {start_station_name}({start_station_id}) => {end_station_name}({end_station_id})")

    route_stations, route_nodes, route_edges = builder.dijkstra(start_station_id, end_station_id, stations_by_id)
    instructions = builder.route_nodes_to_instructions(route_nodes)
    expected_test_result = json.dumps(instructions, separators=(",",":"), indent=None)
    output_test_line = CSHARP_TEMPLATE.format(
      generated_test_index,
      int(start_station_id),
      int(end_station_id),
      start_station_name,
      end_station_name
    )
    generated_test_index += 1
    csharp_output_lines.append(output_test_line)
    possible_routes_output_lines.append(expected_test_result)


In [ ]:
print("\n".join(possible_routes_output_lines))

In [ ]:
print("\n".join(csharp_output_lines))
